<h1 id='parte-3'>4. Extraíndo as informações de uma página web - parte 2</h1>

<h2 id='parte-3a' style="color:blue;">a. Extraindo uma lista de links do google</h2>

<p>Vamos tentar extrair uma lista de notícias do <b>InfoMoney</b> através de uma busca do google na área <b>news</b>, usando como filtros o assunto <b>'itaú'</b> e um intervalo de tempo <b>[01/09/2019, 01/11/2019]</b>. Para isso devemos analisar como o link se comporta de acordo com a nossa pesquisa:</p>

<ul><li><b>url</b> = <b style="color:blue;">https://www.google.com.br/search?q=<b style="color:red;">%22ita%C3%BA%22+infomoney</b>&tbs=cdr:1,<b style="color:red;">cd_min:9/1/2019,cd_max:11/1/2019</b>&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjin7bEyf7lAhWss1kKHRkwAuw4ChD8BSgBegQICxAD&biw=1920&bih=944</b></li></ul>

<p>A primeira parte <b style="color:red;">%22ita%C3%BA%22+infomoney</b> representa nossa pesquisa. Como um link não pode conter <b>aspas(%22)</b> ou letras com acento <b>(ú : %C3%BA)</b>, o link é encodado para um navegador conseguir interpretar o que está sendo passado. A segunda parte <b style="color:red;">cd_min:9/1/2019,cd_max:11/1/2019</b> indica o intervalo de tempo em que estamos fazendo a busca.</p>

<p>Ao mudarmos para a segunda página, vemos que um novo parâmetro aparece dentro do link:</p>

<ul><li><b>url</b> = <b style="color:blue;">https://www.google.com.br/search?q=<b style="color:red;">%22ita%C3%BA%22+infomoney</b>&tbs=cdr:1,<b style="color:red;">cd_min:9/1/2019,cd_max:11/1/2019</b>&tbm=nws&ei=aj_YXeiwGcbZ5gLut4noDw&<b style="color:red;">start=10</b>&sa=N&ved=0ahUKEwjojZbtzv7lAhXGrFkKHe5bAv0Q8tMDCFM&biw=1920&bih=944&dpr=1</b></li></ul>

<p>A parte <b style="color:red;">start=10</b> indica o número da primeira notícia a ser mostrada na página. Cada página apresenta <b>10</b> notícias no total, então a segunda página apresenta as notícias de <b>10 a 19</b>, a terceira <b>20 a 29</b>, etc. Então para facilitar nossa extração de notícias dentro do python, apenas temos que usar links mudando o parâmetro <b style="color:red;">start=0,10,20,30....</b></p>

<p>_______________________________________________________________________________________________________________________</p>

<h3 id='parte-1a1' style="color:green;">a.1. Fazendo um GET na busca e procurando pelos links</h3>

In [1]:
import requests
from bs4 import BeautifulSoup

<p><b>Request</b> no link com parâmetro <b style="color:red;">start=0</b>(primeira página):</p>

In [3]:
url = 'https://www.google.com.br/search?q=%22ita%C3%BA%22+infomoney&tbs=cdr:1,cd_min:9/1/2019,cd_max:11/1/2019&tbm=nws&ei=aj_YXeiwGcbZ5gLut4noDw&start=0&sa=N&ved=0ahUKEwjojZbtzv7lAhXGrFkKHe5bAv0Q8tMDCFM&biw=1920&bih=944&dpr=1'
response = requests.get(url)

response.status_code

200

<p>Organizando a reposta com <b>bs4</b>:</p>

In [4]:
html = BeautifulSoup(response.text, "html.parser")

<p>Para extrair os links de uma página, devemos procurar pelos atributos <b>'href'</b> no código. Inspecionando a página, vemos que ela está dentro de um <b>&lt;div&gt;...&lt;/div&gt;</b> com <b>'class=bkWMgd'</b>, então precisamos passar isso para o <b>'.find_all()'</b></p>

In [5]:
div_search = html.find_all('div', attrs={'class':'bkWMgd'})

div_search

[]

<p>Encontramos um problema, nosso código retornou um vetor vazio, não encontramos o que procurávamos, mesmo inspecionando e sabendo que ele está na página. Mas existe um motivo para isso. Acima do <b>&lt;div&gt;...&lt;/div&gt;</b> há um <b>&lt;script&gt;</b> que diz quando algumas partes da página devem ser geradas ou não. No caso o que queriamos procurar só é gerado depois que o script é acionado, porém com <b>requests.get</b> isso não é possível. Nós apenas estamos pegando o código puro da página usando o <b>GET</b>, então o <b>script</b> nunca é acionado no nosso caso, isso mostrando as limitações do <b>Web Scraping</b>. Em capítulos mais a frente iremos aprender sobre <b>Web Crawling</b> e como contornar essa limitação.</p>

<img src="./imagens/script-google.png" />

<p>Por agora vamos continuar usando as mesmas ferramentas para extrair o que queremos, só teremos um pouco mais de trabalho para isso. Mesmo o <b>&lt;script&gt;</b> não sendo ativado, ainda assim conseguiremos achar os links que queremos(mas não tão organizado como antes).</p>

<p>Vamos procurar então diretamente pela tag <b>&lt;a&gt;...&lt;/a&gt;</b> dentro de todo o código e depois limpamos o que não nos importar:</p>

In [7]:
a_search = html.find_all('a')

a_search

[<a href="/?sa=X&amp;ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQOwgC"><span class="V6gwVd">G</span><span class="iWkuvd">o</span><span class="cDrQ7">o</span><span class="V6gwVd">g</span><span class="ntlR9">l</span><span class="iWkuvd tJ3Myc">e</span></a>,
 <a href="/search?q=%22ita%C3%BA%22+infomoney&amp;tbm=nws&amp;tbs=cdr:1,cd_min:9/1/2019,cd_max:11/1/2019&amp;sa=N&amp;biw=1920&amp;bih=944&amp;ie=UTF-8&amp;gbv=1&amp;sei=wIn_Xf_PCpmf5OUPvs2-2As">aqui</a>,
 <a class="eZt8xd" href="/search?q=%22ita%C3%BA%22+infomoney&amp;biw=1920&amp;bih=944&amp;ie=UTF-8&amp;source=lnms&amp;sa=X&amp;ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQ_AUIBSgA">Todas</a>,
 <a class="eZt8xd" href="/search?q=%22ita%C3%BA%22+infomoney&amp;biw=1920&amp;bih=944&amp;ie=UTF-8&amp;tbm=shop&amp;source=lnms&amp;sa=X&amp;ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQ_AUIBygC">Shopping</a>,
 <a class="eZt8xd" href="https://maps.google.com.br/maps?q=%22ita%C3%BA%22+infomoney&amp;biw=1920&amp;bih=944&amp;dpr=1&amp;um=1&amp;ie=UTF-8&amp;sa=X&amp;ved

<p>Agora para cada <b>&lt;a&gt;...&lt;/a&gt;</b> vamos extrair os <b>'href'</b>:

In [8]:
links = []
for a in a_search:
    link = a.get('href')
    links.append(link)

links

['/?sa=X&ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQOwgC',
 '/search?q=%22ita%C3%BA%22+infomoney&tbm=nws&tbs=cdr:1,cd_min:9/1/2019,cd_max:11/1/2019&sa=N&biw=1920&bih=944&ie=UTF-8&gbv=1&sei=wIn_Xf_PCpmf5OUPvs2-2As',
 '/search?q=%22ita%C3%BA%22+infomoney&biw=1920&bih=944&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQ_AUIBSgA',
 '/search?q=%22ita%C3%BA%22+infomoney&biw=1920&bih=944&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQ_AUIBygC',
 'https://maps.google.com.br/maps?q=%22ita%C3%BA%22+infomoney&biw=1920&bih=944&dpr=1&um=1&ie=UTF-8&sa=X&ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQ_AUICCgD',
 '/search?q=%22ita%C3%BA%22+infomoney&biw=1920&bih=944&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQ_AUICSgE',
 '/search?q=%22ita%C3%BA%22+infomoney&biw=1920&bih=944&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwj_r6Kdx8nmAhWZD7kGHb6mD7sQ_AUICigF',
 '/search?q=%22ita%C3%BA%22+infomoney&biw=1920&bih=944&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahU

<p>_______________________________________________________________________________________________________________________</p>

<h3 id='parte-1a1' style="color:green;">a.2. Limpando o vetor de links encontrado</h3>

<p>Conseguimos extrair os links, mas coletamos mais do que queremos. Devemos então analisar as strings e buscar por padrões dentro do que estão nos links de notícia do <b>InfoMoney</b>. Faremos uma seleção e limpeza no vetor de links procurando por strings com <b>'infomoney'</b> e <b>'/url?q'</b> e que não contenham <b>'accounts.google'</b>. Logo em seguida utilizaremos <b>'.replace()'</b> e <b>'.split()'</b> para limpar o que não faz parte do link das notícias e <b>'set()'</b> para remover qualquer link duplicado.</p>

<p>Exemplo da <b>string</b> que queremos selecionar, em vermelho o que nos interessa nela. Devemos retirar <b style="color:blue;">'/url?q'</b> e tudo o que estiver depois de <b style="color:blue;">'&sa='</b> da string, para assim formar o link:</p>
<p><b style="color:blue;">'/url?q=<b style="color:red;">https://www.infomoney.com.br/mercados/petrobras-corta-preco-do-diesel-em-3-e-itau-compra-startup-por-r-575-mi-4-balancos-e-mais-destaques/</b>&sa=U&ved=2ahUKEwiZjZ-y0_7lAhVu7HMBHTUsCzQQxfQBMAV6BAgGEAE&usg=AOvVaw2LQIjB7DGKFpTp4u0FMw8j'</b></p>

In [9]:
links_im = []
for i in range(len(links)):
    if ('infomoney' in links[i]) and ('/url?q' in links[i]) and ('accounts.google' not in links[i]):
        links_im.append(links[i].replace('/url?q=', '').split('&sa=')[0])
        
links_im = list(set(links_im))

links_im

['https://www.infomoney.com.br/negocios/rede-amplia-pagamento-em-2-dias-sem-taxa-para-clientes-sem-conta-no-itau-cielo-despenca/',
 'https://www.infomoney.com.br/mercados/itau-unibanco-soma-mais-de-r-45-bi-em-transacoes-e-bate-recorde-com-black-friday/',
 'https://www.infomoney.com.br/negocios/com-dinheiro-de-abertura-de-capital-vivara-quer-abrir-50-lojas-em-2020/',
 'https://www.infomoney.com.br/mercados/as-acoes-que-mais-pagaram-dividendos-no-ano-e-as-promessas-para-2020/',
 'https://www.infomoney.com.br/mercados/banco-do-brasil-e-itau-aprovam-pagamento-de-juros-sobre-capital-proprio-aos-acionistas/',
 'https://www.infomoney.com.br/mercados/acao-da-via-varejo-tem-nova-alta-de-5-vale-e-csn-avancam-com-dados-positivos-chineses/',
 'https://www.infomoney.com.br/minhas-financas/itau-e-bradesco-seguem-copom-e-cortam-juros-de-linhas-de-credito/',
 'https://www.infomoney.com.br/negocios/guerra-das-maquininhas-cade-rejeita-recurso-do-itau-e-mantem-suspensao-de-promocao-da-rede/',
 'https://w

<p>_______________________________________________________________________________________________________________________</p>

<h3 id='parte-1a1' style="color:green;">a.3. Extraindo links de várias páginas</h3>

<p>Em <b style="color:green";>a.2.</b> conseguimos extrair todos os links da primeira página da nossa busca do google. Podemos estender nosso código para pegar <b>n</b> páginas. Faremos como exemplo <b>n = 4</b>, ou seja, iremos até a página 4 usando como parâmetro <b>[0, 10, 20, 30]</b> no link, com apenas um detalhe a mais: Iremos retirar links <b>/patrocinados/</b> da nossa busca.

In [10]:
# Parâmetros da nossa busca
pag = [0, 10, 20, 30]
busca = '%22ita%C3%BA%22+infomoney'
per = 'cd_min:9/1/2019,cd_max:11/1/2019'

# Vetor para armazenar os links do InfoMoney
links_im = []

# Loop entre as páginas de 1 a 4
for n in pag:
    # Request da página
    url = f'https://www.google.com.br/search?q={busca}&tbs=cdr:1,{per}&tbm=nws&ei=aj_YXeiwGcbZ5gLut4noDw&start={n}&sa=N&ved=0ahUKEwjojZbtzv7lAhXGrFkKHe5bAv0Q8tMDCFM&biw=1920&bih=944&dpr=1'
    response = requests.get(url)
    
    # HTML organizado pelo bs4
    html = BeautifulSoup(response.text, "html.parser")
    
    # Extração de todos os <a>...</a>
    a_search = html.find_all('a')
    
    # Retirando os 'href'
    links = []
    for a in a_search:
        link = a.get('href')
        links.append(link)
    
    # Selecionando os links de notícias do InfoMoney
    for i in range(len(links)):
        if ('infomoney' in links[i]) and ('/url?q' in links[i]) and ('accounts.google' not in links[i]) and ('patrocinados' not in links[i]):
            links_im.append(links[i].replace('/url?q=', '').split('&sa=')[0])

# Retirando links duplicados
links_im = list(set(links_im))

links_im

['https://www.infomoney.com.br/mercados/acao-da-hypera-sobe-forte-apos-compra-da-marca-buscopan-marfrig-cai-5-com-oferta-de-acoes/',
 'https://www.infomoney.com.br/negocios/com-dinheiro-de-abertura-de-capital-vivara-quer-abrir-50-lojas-em-2020/',
 'https://www.infomoney.com.br/mercados/as-acoes-que-mais-pagaram-dividendos-no-ano-e-as-promessas-para-2020/',
 'https://www.infomoney.com.br/mercados/acao-da-via-varejo-sobe-na-expectativa-pela-black-friday-bancos-caem-apos-decisao-do-cmn/',
 'https://www.infomoney.com.br/minhas-financas/fintechs-ajudam-a-reduzir-custo-para-envio-de-dinheiro-ao-exterior-conheca/',
 'https://www.infomoney.com.br/mercados/tres-noticias-sobre-petrobras-black-rock-eleva-fatia-na-vale-e-mais-destaques/',
 'https://www.infomoney.com.br/negocios/mesmo-apos-subir-156-via-varejo-convence-analistas-com-discurso-de-reestruturacao/',
 'https://www.infomoney.com.br/mercados/tst-autoriza-petrobras-a-suspender-repasse-a-sindicato-em-meio-a-greve-marisa-anuncia-oferta-de-ac